In [1]:
import pypdf

reader = pypdf.PdfReader("../PDFFiles/Course.pdf")


reader.pages[22].extract_text()

' \nB.E.(CSE) – Degree Programme 2022 - 2023 \n \nApproved in 63rd Academic Council meeting on 25.06.2022  \n                  \n \n \n \nAssessment Pattern \nCO CAT1 CAT2 Terminal \n   \nTPS \nScale 1 2 3 4 5 6 1 2 3 4 5 6 1 2 3 4 5 6 \nCO1 4 20 0                   2 8      \nCO2 4 0 20                   2 4 10    \nCO3 4 20 0                2 8      \nCO4 8 0 20                 2 4 10    \nCO5             12 20 20       6 8 10    \nCO6             8 20 20       6 8 10    \n \n*Terminal examination should cover all Course Outcomes in the appropriate TPS Scale level. \n \nCO Assignment 1* Assignment 2* \n  \nTPS \nScale \n1 2 3 4 5 6 1 2 3 4 5 6 \nCO1             \nCO2   20          \nCO3             \nCO4   20          \nCO5         20    \nCO6                 20       \n   *Assessment type: Quiz / Test /Presentation \n \nSyllabus \nWater: Water-sources- physical - characteristics - alkalinity - hardness of water – types -\ndetermination of hardness by EDTA method. Boiler trouble -Sof

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai") 

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--592a3e0b-1473-4f2e-8c6a-7062a3c964f4-0', usage_metadata={'input_tokens': 5, 'output_tokens': 11, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}})

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [22]:
from langchain_community.document_loaders import PyPDFLoader

filePath = "./Course.pdf"
loader = PyPDFLoader(file_path=filePath)

docs = loader.load()

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)


In [24]:
_ = vector_store.add_documents(documents=all_splits)

In [25]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [26]:
from typing_extensions import TypedDict, List
from langgraph.graph import START, StateGraph
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = model.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [33]:
response = graph.invoke({
    "question" : "what are the course outcomes of Database Management System course"
})

response["answer"]

'The course outcomes of the Database Management System course are: to develop data models, manipulate relational databases, design normalized databases, construct data structures for fast data retrieval, and make use of different forms of transactions, concurrency control, and recovery. Students will also be able to create Entity Relationship and Relational models for real-world applications. These outcomes aim to provide students with a comprehensive understanding of database management systems.'